In [1]:
import csv, pandas as pd  ,  datetime

fname = "/home/jl/Dropbox/Python Stuff/01_Data/activities.csv"

df = pd.read_csv(fname, header=0)

df = df[["Name", 'Type', 'When', 'Dist km', 'Elv m', 'Elapsed Time', 'Moving Time', 'Heart', 'Cal', 'Ride/Run Type', 'Relative Effort', 'Points in red', 'Activity Id']]

df["dist_km"] = df["Dist km"] /1000
df['date'] = pd.to_datetime(df['When']).dt.normalize()
df['year'] = df['date'].apply(lambda x: x.strftime('%Y'))

In [2]:
replace_dict = {
    "Name": "activity_name",     'Type': 'type', 'When': 'datetime', 'Elapsed Time': 'time_elapsed', 
'Moving Time': 'time_moving', 'Heart': 'heart_average', 'Cal': 'calories', 'Ride/Run Type': 'type-sub', 
'Relative Effort': 'relative_effort', 'Points in red': 'points_in_red', 'Activity Id': 'activity_id', 
'dist_comb': 'eqkm_distance', 'ascent_comb': 'eqkm_ascent', 'date': 'date', 
'date_str': 'date_str', 'x_date': 'x_date', 'year': 'year','Elv m': 'elevation_metres', 
}
#normalise the names from replace_dict
df = df.rename(columns=replace_dict)
#set index
df = df.set_index('datetime')

In [5]:
#get the best elevation factor for eqkm

from sklearn.linear_model import LinearRegression

#just get the runs with more than 100 metres elevation
df3 = df[(df.elevation_metres > 100) & (df.type == 'Run') ].copy()

best_r_squared = 0
best_i = 0

# loop through and find the factor that best explains the moving times, that with the best r squared 
for i in range (1, 200):
    df3["eqkm"] = df3["dist_km"] + (df3["elevation_metres"] / i)
    model = LinearRegression()
    X, y = df3[['eqkm','year']], df3['time_moving']
    model.fit(X, y)
    r_squared = model.score(X, y)
  
    if r_squared > best_r_squared:
        best_i = i
        best_r_squared = r_squared
    
print("best_r_squared is " +str(  round(best_r_squared ,3)             ))
print(str(best_i) + " metres is equivalent to 1 km flat")
df["eqkm"] = df["dist_km"] + (df["elevation_metres"] / best_i)

df["eqkm_ascent"] =  (df["elevation_metres"] / best_i)


best_r_squared is 0.951
64 metres is equivalent to 1 km flat


In [7]:
df.to_csv("/home/jl/Dropbox/Python Stuff/01_Data/output_veloviewer_run_cumulative.csv", sep=';', encoding='utf-8', index = False)